In [15]:
from prod.set_seed import random_ctl
seed = random_ctl()

from fastai.text import *
from fastai.callbacks import SaveModelCallback
from fastai.layers import LabelSmoothingCrossEntropy

import sentencepiece as spm #https://github.com/google/sentencepiece
import fire

from prod.sp_tok import *
from prod.bin_metrics import Fbeta_binary

def split_rebal_data_by_idx(all_texts_df:DataFrame, train_idx, valid_idx, 
                             clas_col:str='is_humor'):
    ## rebalance cases
    print('Number of positive samples:', (all_texts_df.loc[:,clas_col] == 1).sum())
    print('Number of negative samples:',  (all_texts_df.loc[:,clas_col] == 0).sum())
    print('Total samples:', len(all_texts_df))

    df_train_all = all_texts_df.iloc[train_idx,:]
    df_valid     = all_texts_df.iloc[valid_idx,:]

    print('Valid prevalence(n = %d):'%len(df_valid),df_valid.loc[:,clas_col].sum()/ len(df_valid))
    print('Train all prevalence(n = %d):'%len(df_train_all), df_train_all.loc[:,clas_col].sum()/ len(df_train_all))
    print('all samples (n = %d)'%len(all_texts_df))
    assert len(all_texts_df) == (len(df_valid)+len(df_train_all)),'math didnt work'

    ## assumes that negative is less frequent case.  Generalize?
    rows_pos = df_train_all.loc[:,clas_col] == 1
    df_train_pos = df_train_all.loc[rows_pos]
    df_train_neg = df_train_all.loc[~rows_pos]
    print(f'Train Pos Cases:{df_train_pos.shape},Train Neg Cases:{df_train_neg.shape}')
    df_resample_pos = df_train_pos.sample(n=len(df_train_neg),replace=True,
                                          axis=0,random_state=np.random.get_state()[1][0]).reset_index(drop=True)
    df_train = pd.concat([df_resample_pos,df_train_neg],axis=0) #randomized again in DataBunch?
    print('Train prevalence (n = %d):'%len(df_train), df_train.loc[:,clas_col].sum()/ len(df_train))

    return df_train, df_valid

Using seed: 784007


In [16]:
model_path = './data/rnn'
sp_model = 'all_tweets_es_0606'
# batch_size = 64
backward = 0
# dropmult = 0.5
qrnn = True
n_hid:int=2304
flat_loss = True
split_seed = 20190313
# enc_name = 'twitter_es_enc_QRNN_0608_drop1_seed0_finetune'

In [17]:
PATH = Path(model_path)

defaults.text_spec_tok.append(NL) #add a New Line special char
sp_vocab = Vocab( get_itos(sp_model) )
mycust_tok = CustomTokenizer(SPTokenizer, sp_model, pre_rules=default_rules)

all_texts_df = pd.read_csv('./data/haha_2019_train.csv')
raw_text = all_texts_df.loc[:,'text']

print("Default Rules:\n",[x.__name__ for x in default_rules],"\n\n")
for rule in default_rules: raw_text = raw_text.apply(lambda x: rule(str(x)))
all_texts_df['new_text'] = raw_text #databunch adds `xxbos` so don't add here

Default Rules:
 ['fixup', 'replace_rep', 'replace_wrep', 'deal_caps', 'spec_add_spaces', 'rm_useless_spaces', 'sub_nl'] 




In [18]:
from sklearn.model_selection import KFold
all_texts_df.shape
kfolder = KFold(n_splits=5, random_state=split_seed, shuffle=True)

In [19]:
for n_fold, (train_idx,valid_idx) in enumerate(kfolder.split(all_texts_df)):
    print(n_fold, train_idx.shape, valid_idx.shape)

0 (19200,) (4800,)
1 (19200,) (4800,)
2 (19200,) (4800,)
3 (19200,) (4800,)
4 (19200,) (4800,)


In [14]:
df_train,df_valid = split_rebal_data_by_idx(all_texts_df, train_idx, valid_idx)

Number of positive samples: 9253
Number of negative samples: 14747
Total samples: 24000
Valid prevalence(n = 4800): 0.39375
Train all prevalence(n = 19200): 0.38348958333333333
all samples (n = 24000)
Train Pos Cases:(7363, 11),Train Neg Cases:(11837, 11)
Train prevalence (n = 23674): 0.5
